# Importing and preparing rental apartments data

## Libraries and settings

In [1]:
# Libraries
import os
import re
import fnmatch
import datetime
import numpy as np
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Show current working directory
print(os.getcwd())

/workspaces/data_analytics/Week_03


## Importing data

In [2]:
# Read the data to a pandas data frame
df = pd.read_csv('apartments_data_zuerich.csv', sep=',', encoding='utf-8')

# Get number of rows and columns
df.shape

(1008, 7)

## Count number of rows and columns in the data frame

In [3]:
# Dimension (rows, columns)
print('Dimension:', df.shape)

# Number of rows
print('Number of rows:', df.shape[0])

# Number of columns
print('Number of columns:', df.shape[1])

Dimension: (1008, 7)
Number of rows: 1008
Number of columns: 7


## Get data types (raw-format from web scraping)

In [4]:
# Get data types (note that in pandas, a string is referred to as 'object')
df.dtypes

web-scraper-order        object
web-scraper-start-url    object
rooms_area_price_raw     object
address_raw              object
price_raw                object
description_raw          object
text_raw                 object
dtype: object

## Extract and save relevant information from raw data using regular expressions (regex)

### Extract number of rooms

In [5]:
# Extract values from 'rooms_area_price_raw' strings
rooms = []
for i in df['rooms_area_price_raw']:
    d1 = re.findall('(.*)Zimmer', i)
    try:
        d2 = d1[0].strip().replace(',', '.')
    except:
        d2 = None
    rooms.append(d2)

# Save as new variable in the pandas data frame
df['rooms'] = pd.Series(rooms, dtype="float64")
    
# Print first 5 values
print(df['rooms_area_price_raw'].head(5), '\n')
print(df['rooms'].head(5), '\n')

0      3 Zimmer, 49 m², CHF 1441.—
1    3,5 Zimmer, 65 m², CHF 1850.—
2                19 m², CHF 2686.—
3      2 Zimmer, 54 m², CHF 4853.—
4      2 Zimmer, 49 m², CHF 4335.—
Name: rooms_area_price_raw, dtype: object 

0    3.0
1    3.5
2    NaN
3    2.0
4    2.0
Name: rooms, dtype: float64 



### Extract living area

In [6]:
# Extract values from 'rooms_area_price_raw' strings
area = []
for i in df['rooms_area_price_raw']:
    d1 = re.findall('Zimmer, (.*)m²', i)
    try:
        d2 = d1[0].strip()
    except:
        d2 = None
    area.append(d2)

# Save as new variable in the pandas data frame
df['area'] = pd.Series(area, dtype="float64")

# Print first 5 values
print(df['rooms_area_price_raw'].head(5), '\n')
print(df['area'].head(5), '\n')

0      3 Zimmer, 49 m², CHF 1441.—
1    3,5 Zimmer, 65 m², CHF 1850.—
2                19 m², CHF 2686.—
3      2 Zimmer, 54 m², CHF 4853.—
4      2 Zimmer, 49 m², CHF 4335.—
Name: rooms_area_price_raw, dtype: object 

0    49.0
1    65.0
2     NaN
3    54.0
4    49.0
Name: area, dtype: float64 



### Extract rental price

In [7]:
# Extract values from 'price_raw' strings
price = []
for i in df['price_raw']:
    d1 = re.findall('[0-9]+', i)
    try:
        d2 = d1[0].strip()
    except:
        d2 = None
    price.append(d2)

# Save as new variable in the pandas data frame
df['price'] = pd.Series(price, dtype="float64")

# Print first 5 values
print(df['price_raw'].head(), '\n')
print(df['price'].head())

0    CHF 1441.—
1    CHF 1850.—
2    CHF 2686.—
3    CHF 4853.—
4    CHF 4335.—
Name: price_raw, dtype: object 

0    1441.0
1    1850.0
2    2686.0
3    4853.0
4    4335.0
Name: price, dtype: float64


## Create additional variables from the apartment's descriptions

### Change strings in 'description_raw' ad 'text_raw' to uppercase 

In [8]:
# Change strings to uppercase 
df['description_raw'] = df['description_raw'].str.upper()
print(df['description_raw'].head(10), '\n')

df['text_raw'] = df['text_raw'].str.upper()
print(df['text_raw'].head(10))

0                   «GEMÜTLICHE WOHNUNG IM GRÜNEN»
1    «ATTRAKTIVE 3.5-ZIMMER-EG-WOHNUNG IN NEERACH»
2                «STUDIO APARTMENT JUNIOR BALCONY»
3             «2 BEDROOM APARTMENT SENIOR BALCONY»
4             «2 BEDROOM APARTMENT JUNIOR TERRACE»
5                «STUDIO APARTMENT SENIOR TERRACE»
6             «1 BEDROOM APARTMENT JUNIOR TERRACE»
7                     «2 BEDROOM APARTMENT JUNIOR»
8                        «STUDIO APARTMENT JUNIOR»
9                     «1 BEDROOM APARTMENT JUNIOR»
Name: description_raw, dtype: object 

0    3 ZIMMER, 49 M², CHF 1441.—NEUHUSSTRASSE 6, 86...
1    3,5 ZIMMER, 65 M², CHF 1850.—ZÜRCHERSTRASSE 1,...
2    19 M², CHF 2686.—CRAMERSTRASSE 8-12, 8004 ZÜRI...
3    2 ZIMMER, 54 M², CHF 4853.—CRAMERSTRASSE 8-12,...
4    2 ZIMMER, 49 M², CHF 4335.—ROTACHSTRASSE 33, 8...
5    34 M², CHF 3205.—BINZMÜHLESTR. 50, 8050 ZÜRICH...
6    25 M², CHF 3308.—ROTACHSTRASSE 33, 8003 ZÜRICH...
7    43 M², CHF 4241.—MILITÄRSTRASSE 24, 8004 ZÜRIC...
8    19 M²,

### Calculate lenght of strings in 'description_raw' and 'text_raw'

In [9]:
# Show first item of variable 'description_raw'
print(df['description_raw'][0])

# Lenght of the strings in 'description_raw'
df['description_raw_len'] = df['description_raw'].str.len()
print(df['description_raw_len'], '\n')

# Show first item of variable 'text_raw'
print(df['text_raw'][0])

# Lenght of the strings in 'text_raw'
df['text_raw_len'] = df['text_raw'].str.len()
print(df['text_raw_len'])

«GEMÜTLICHE WOHNUNG IM GRÜNEN»
0       30
1       45
2       33
3       36
4       36
        ..
1003    47
1004    47
1005    56
1006    33
1007    40
Name: description_raw_len, Length: 1008, dtype: int64 

3 ZIMMER, 49 M², CHF 1441.—NEUHUSSTRASSE 6, 8630 RÜTI ZH, ZH«GEMÜTLICHE WOHNUNG IM GRÜNEN»WIR VERMIETEN DIESE GEMÜTLICHE 3-ZIMMERWOHNUNG IN RÜTI ZH.DIE WOHNUNG ÜBERZEUGT MIT FOLGENDEN ECKDATEN:- KÜCHE: SANIERT 2012 MIT MODERNEM AUSBAUSTANDART, GESCHIRRSPÜLER, BACKOFEN, GLASKERAMIKHERD, GROSSER KÜHLSCHRANK- BAD: SANIERT 2012, MODERNER, SCHLICHTER STIL MIT BADEWANNE UND FENSTER- BALKON: MIT SICHT INS GRÜNE- KORRIDOR: PLATTENBELAG UND PLATZ FÜR GARDEROBE- WOHNRÄUME: LAMINAT UND FENSTERFRONT MIT VIEL LICHT- FENSTER/FASSADE UND BALKONE 2023 FRISCH SANIERT- KELLER- UND ESTRICHABTEIL- WASCHKÜCHE ZUR MITBENUTZUNG- FENSTER, BALKONE UND FASSA
0       640
1       658
2       682
3       695
4       693
       ... 
1003    668
1004    679
1005    712
1006    653
1007    655
Name: text_raw_len,

### Create new binary (0/1) variable 'luxurious' from the description

In [10]:
# Create a pattern which can be used to search the variable 'description_raw'
pattern = 'LOFT|SEESICHT'

# Create new variable 'luxurious' as binary dummy (0/1) variable
df['luxurious'] = df['description_raw'].str.contains(pat = pattern).astype(int)
print(df['luxurious'].sum())

# Show values
df[['description_raw','rooms','area','price','luxurious']]

14


,description_raw,rooms,area,price,luxurious
0,«GEMÜTLICHE WOHNUNG IM GRÜNEN»,3.0,49.0,1441.0,0
1,«ATTRAKTIVE 3.5-ZIMMER-EG-WOHNUNG IN NEERACH»,3.5,65.0,1850.0,0
2,«STUDIO APARTMENT JUNIOR BALCONY»,NaN,NaN,2686.0,0
3,«2 BEDROOM APARTMENT SENIOR BALCONY»,2.0,54.0,4853.0,0
4,«2 BEDROOM APARTMENT JUNIOR TERRACE»,2.0,49.0,4335.0,0
...,...,...,...,...,...
1003,"«CHARMANTE WOHNUNG IN HOTTINGEN, NÄHE RÖMERHOF»",3.5,82.0,2830.0,0
1004,«HEIMELIGE 4-ZIMMER-WOHNUNG IN WINTERTHUR-HEGI»,4.0,73.0,1713.0,0
1005,«EXKLUSIVE WOHNUNG FÜR HÖCHSTE ANSPRÜCHE - MIT...,4.5,110.0,3500.0,1
1006,«MODERNE WOHNUNG AN RUHIGER LAGE»,3.0,68.0,1650.0,0


### Create new binary (0/1) variable 'furnished' from the description

In [33]:
# Create a pattern which can be used to search the variable 'description_raw'
pattern = '(MÖBLIERT)'

# Create new variable 'luxurious' as binary dummy (0/1) variable
df['furnished'] = df['description_raw'].str.contains(pat = pattern).astype(int)
print(df['furnished'].sum())

# Filter only furnished apartments
df_furnished = df[df['furnished'] == 1]

# Show values only for furnished apartments
df_furnished[['description_raw','rooms','area','price','furnished']]

38


,description_raw,rooms,area,price,furnished
49,"«MÖBLIERT, TEMPORÄR: 2½ ZI-WOHNUNG IN GLATTPAR...",2.5,78.0,2750.0,1
77,"«MÖBLIERT, TEMPORÄR: 2½ ZI-WOHNUNG IN ZÜRICH -...",2.5,45.0,2520.0,1
78,«MÖBLIERTES ZIMMER (NEU!)»,1.0,18.0,1100.0,1
85,«MÖBLIERTES GROSSES ZIMMER IN WEININGEN ZH»,1.0,30.0,500.0,1
86,"«MÖBLIERT, TEMPORÄR: 2 ZI-WOHNUNG IN ZÜRICH - ...",2.0,70.0,1625.0,1
88,«MODERN MÖBLIERTES WG-ZIMMER IN BÜLACH»,1.0,20.0,750.0,1
89,"«MÖBLIERT, TEMPORÄR: 4½ ZI-WOHNUNG IN WINTERTH...",4.5,117.0,3782.0,1
145,«LUXURIÖS MÖBLIERTE 3.0 ZI-WHG MIT EINZIGARTIG...,3.0,100.0,4800.0,1
147,«MÖBLIERT: 2½ ZI-WOHNUNG IN ZÜRICH - KREIS 4»,2.5,48.0,3360.0,1
149,«MÖBLIERTE 4.5 ZIMMER-WOHNUNG MIT TRAUMHAFTER ...,4.5,104.0,2800.0,1


### Create new binary (0/1) variable 'balcony' from the description

In [32]:
# Create a pattern which can be used to search the variable 'description_raw'
pattern = '(BALKON) |(TERRASSE)'

# Create new variable 'luxurious' as binary dummy (0/1) variable
df['balcony'] = df['description_raw'].str.contains(pat = pattern).astype(int)
print(df['balcony'].sum())

# Filter only apartments with balcony
df_balcony = df[df['balcony'] == 1]

# Show values only for apartments with balcony
df_balcony[['description_raw','rooms','area','price','balcony']]

50


,description_raw,rooms,area,price,balcony
24,«MODERNE 3.5-ZIMMER-NEUBAUWOHNUNG MIT BALKON U...,3.5,105.0,3200.0,1
44,«CHARMANTE WOHNUNG MIT TRAUMHAFTER TERRASSE»,3.5,80.0,2237.0,1
66,«SCHÖNE 3.5-ZIMMERWOHNUNG MIT BALKON ZU VERMIE...,3.5,80.0,1980.0,1
167,«MODERNE 5.5-ZIMMER-WOHNUNG MIT GROSSER TERRAS...,5.5,140.0,2850.0,1
168,«ERSTBEZUG NACH SANIERUNG - WOHNUNG MIT BALKON...,1.5,33.0,1440.0,1
170,«MAISONETTE-DACHWOHNUNG MIT TERRASSE IM EHEMAL...,3.5,75.0,1740.0,1
243,«MIT GROSSZÜGIGER TERRASSE»,4.5,140.0,2750.0,1
303,«TOP ZUSTAND! TOP LAGE! 4.5 ZIMMERWOHNUNG MIT ...,4.5,97.0,2590.0,1
379,«ZENTRAL GELEGENE WOHNUNG MIT BALKON 31.03.2024»,3.0,63.0,1465.0,1
393,«SANIERTE 3.5-ZIMMER-WOHNUNG MIT TERRASSE»,3.5,100.0,4950.0,1


### Create new binary (0/1) variable 'central' from the description

In [30]:
# Create a pattern which can be used to search the variable 'description_raw'
pattern = '(ZENTRAL)'

# Create new variable 'luxurious' as binary dummy (0/1) variable
df['central'] = df['description_raw'].str.contains(pat = pattern).astype(int)
print(df['central'].sum())

# Filter only furnished apartments
df_furnished = df[df['central'] == 1]

# Show values only for furnished apartments
df_furnished[['description_raw','rooms','area','price','central']]

70


,description_raw,rooms,area,price,central
53,«WOHNUNG AN ZENTRALER LAGE IM ANDREASPARK»,4.5,141.0,3309.0,1
57,"«1.5-ZIMMER-WOHNUNG AN RUHIGER, ZENTRALER LAGE»",1.5,35.0,1740.0,1
58,"«VIEL RAUM, RENOVIERT UND ZENTRAL»",4.0,80.0,2150.0,1
67,«ZENTRALES LOFTLEBEN NEUES DESIGN»,NaN,NaN,2590.0,1
95,«MODERNES STUDIO AN ZENTRALER LAGE ZU VERMIETEN!»,1.0,25.0,1290.0,1
...,...,...,...,...,...
874,«ZENTRALE LAGE IN WÜLFLINGEN»,2.0,56.0,1360.0,1
883,«SEHR SCHÖNE WOHNUNG AN ZENTRALER LAGE»,4.5,96.0,2245.0,1
887,«GROSSZÜGIGE 1.5-ZIMMERWOHNUNG RUHIG UND DOCH ...,1.5,38.0,1890.0,1
905,«ZENTRALES WOHNEN IN USTER!»,3.5,66.0,1568.0,1


### Create new categorical variable based on apartment area

In [11]:
# Define classes (labels)
labels = ['0 - 49', '50 - 99', '100 - 9999']

# Use the .cut method from pandas to divide the numeric values in classes
df["area_cat"] = pd.cut(df['area'], bins=[0, 50, 100, 10000], labels=labels)

# Show original data and classes
df[['area', 'area_cat']]

,area,area_cat
0,49.0,0 - 49
1,65.0,50 - 99
2,NaN,NaN
3,54.0,50 - 99
4,49.0,0 - 49
...,...,...
1003,82.0,50 - 99
1004,73.0,50 - 99
1005,110.0,100 - 500
1006,68.0,50 - 99


### Create new numeric variable 'price_per_m2'

In [36]:
# Create the new variable
df['price_per_m2'] = round(df['price'] / df['area'], 2)

# Show values
# df[['description_raw','rooms','area','price','luxurious','price_per_m2']]

# Show values for apartments with the highest price per m2
df[['description_raw','rooms','area','price','luxurious','price_per_m2']].sort_values(by='price_per_m2', ascending=False).head(10)

,description_raw,rooms,area,price,luxurious,price_per_m2
657,"«STUDIO, ZÜRICH ZENTRUM, UNBEFRISTET»",1.0,10.0,1499.0,0,149.90
82,«HIGH QUALITY SERVICED STUDIO APARTMENT IN SEE...,1.5,25.0,3490.0,0,139.60
620,«SENIOR 1.5 ROOM APARTMENT WITH BALCONY»,1.5,30.0,3890.0,0,129.67
306,«LIVING THE EXTRAORDINARY! UPSCALE 1-BEDROOM A...,2.5,65.0,8290.0,0,127.54
81,«ERFRISCHEND & FARBENFROH - LANGEWEILE ADIÒS! ...,1.5,30.0,3790.0,0,126.33
15,«2 BEDROOM APARTMENT MINI»,2.0,32.0,3515.0,0,109.84
433,«KLEINES ZIMMER MIT KÜCHE UND BAD - NÄHE ZÜRIC...,1.0,11.0,1120.0,0,101.82
584,«MODERN MÖBILIERTE GEMÜTLICHE 1-ZIMMERWOHNUNG ...,1.0,15.0,1500.0,0,100.00
424,«WOW AN DER LIMMAT»,1.5,22.0,2150.0,0,97.73
192,«2 BEDROOM APARTMENT JUNIOR»,2.0,41.0,3930.0,0,95.85


### Create new categorical variable based on 'price_per_m2'

In [38]:
# Define classes (labels)
labels = ['low', 'medium', 'high']

# Use the .cut method from pandas to divide the numeric values in classes
df["price_cat"] = pd.cut(df['price_per_m2'], bins=[0, 50, 100, 10000], labels=labels)

# Show original data and classes
df[['price_per_m2', 'price_cat']]

,price_per_m2,price_cat
0,29.41,low
1,28.46,low
2,NaN,NaN
3,89.87,medium
4,88.47,medium
...,...,...
1003,34.51,low
1004,23.47,low
1005,31.82,low
1006,24.26,low


### Including current datetime

In [13]:
# Get and format datetime
df['datetime'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Show values
df[['description_raw','rooms','area','price','luxurious','price_per_m2', 'datetime']]

,description_raw,rooms,area,price,luxurious,price_per_m2,datetime
0,«GEMÜTLICHE WOHNUNG IM GRÜNEN»,3.0,49.0,1441.0,0,29.41,2024-10-10 17:20:36
1,«ATTRAKTIVE 3.5-ZIMMER-EG-WOHNUNG IN NEERACH»,3.5,65.0,1850.0,0,28.46,2024-10-10 17:20:36
2,«STUDIO APARTMENT JUNIOR BALCONY»,NaN,NaN,2686.0,0,NaN,2024-10-10 17:20:36
3,«2 BEDROOM APARTMENT SENIOR BALCONY»,2.0,54.0,4853.0,0,89.87,2024-10-10 17:20:36
4,«2 BEDROOM APARTMENT JUNIOR TERRACE»,2.0,49.0,4335.0,0,88.47,2024-10-10 17:20:36
...,...,...,...,...,...,...,...
1003,"«CHARMANTE WOHNUNG IN HOTTINGEN, NÄHE RÖMERHOF»",3.5,82.0,2830.0,0,34.51,2024-10-10 17:20:36
1004,«HEIMELIGE 4-ZIMMER-WOHNUNG IN WINTERTHUR-HEGI»,4.0,73.0,1713.0,0,23.47,2024-10-10 17:20:36
1005,«EXKLUSIVE WOHNUNG FÜR HÖCHSTE ANSPRÜCHE - MIT...,4.5,110.0,3500.0,1,31.82,2024-10-10 17:20:36
1006,«MODERNE WOHNUNG AN RUHIGER LAGE»,3.0,68.0,1650.0,0,24.26,2024-10-10 17:20:36


## Count, identify and remove missing values

In [14]:
# Count missing values
print('Count missing values per variable')
print(pd.isna(df).sum(), '\n')

# Identify rows with missing values
print('Identify rows with missing values')
print(df.loc[df.isna().any(axis=1)][['rooms', 'area', 'price']], '\n')

# Drop rows where at least one element is missing.
df2 = df.dropna()
df2.head()

Count missing values per variable
web-scraper-order          0
web-scraper-start-url      0
rooms_area_price_raw       0
address_raw                0
price_raw                  0
description_raw            0
text_raw                   0
rooms                     45
area                     135
price                     15
description_raw_len        0
text_raw_len               0
luxurious                  0
area_cat                 135
price_per_m2             143
datetime                   0
dtype: int64 

Identify rows with missing values
     rooms  area   price
2      NaN   NaN  2686.0
5      NaN   NaN  3205.0
6      NaN   NaN  3308.0
7      NaN   NaN  4241.0
8      NaN   NaN  2582.0
..     ...   ...     ...
929    3.5   NaN  1910.0
956    4.5   NaN  2000.0
957    6.5   NaN  3200.0
970    1.0   NaN  1350.0
993    3.5   NaN  3087.0

[143 rows x 3 columns] 



,web-scraper-order,web-scraper-start-url,rooms_area_price_raw,address_raw,price_raw,description_raw,text_raw,rooms,area,price,description_raw_len,text_raw_len,luxurious,area_cat,price_per_m2,datetime
0,1693998201-1,https://www.immoscout24.ch/de/immobilien/miete...,"3 Zimmer, 49 m², CHF 1441.—","Neuhusstrasse 6, 8630 Rüti ZH, ZH",CHF 1441.—,«GEMÜTLICHE WOHNUNG IM GRÜNEN»,"3 ZIMMER, 49 M², CHF 1441.—NEUHUSSTRASSE 6, 86...",3.0,49.0,1441.0,30,640,0,0 - 49,29.41,2024-10-10 17:20:36
1,1693998201-2,https://www.immoscout24.ch/de/immobilien/miete...,"3,5 Zimmer, 65 m², CHF 1850.—","Zürcherstrasse 1, 8173 Neerach, ZH",CHF 1850.—,«ATTRAKTIVE 3.5-ZIMMER-EG-WOHNUNG IN NEERACH»,"3,5 ZIMMER, 65 M², CHF 1850.—ZÜRCHERSTRASSE 1,...",3.5,65.0,1850.0,45,658,0,50 - 99,28.46,2024-10-10 17:20:36
3,1693998201-4,https://www.immoscout24.ch/de/immobilien/miete...,"2 Zimmer, 54 m², CHF 4853.—","Cramerstrasse 8-12, 8004 Zürich, ZH",CHF 4853.—,«2 BEDROOM APARTMENT SENIOR BALCONY»,"2 ZIMMER, 54 M², CHF 4853.—CRAMERSTRASSE 8-12,...",2.0,54.0,4853.0,36,695,0,50 - 99,89.87,2024-10-10 17:20:36
4,1693998201-5,https://www.immoscout24.ch/de/immobilien/miete...,"2 Zimmer, 49 m², CHF 4335.—","Rotachstrasse 33, 8003 Zürich, ZH",CHF 4335.—,«2 BEDROOM APARTMENT JUNIOR TERRACE»,"2 ZIMMER, 49 M², CHF 4335.—ROTACHSTRASSE 33, 8...",2.0,49.0,4335.0,36,693,0,0 - 49,88.47,2024-10-10 17:20:36
15,1693998201-16,https://www.immoscout24.ch/de/immobilien/miete...,"2 Zimmer, 32 m², CHF 3515.—","Wolframplatz 1, 8045 Zürich, ZH",CHF 3515.—,«2 BEDROOM APARTMENT MINI»,"2 ZIMMER, 32 M², CHF 3515.—WOLFRAMPLATZ 1, 804...",2.0,32.0,3515.0,26,681,0,0 - 49,109.84,2024-10-10 17:20:36


## Count, identify & remove duplicated values

In [15]:
# Count duplicated values in the whole data set
print('Sum of missing values:', df.duplicated().sum(), '\n')

# Identify duplicated values in 'rooms', 'area', 'price'
print('Duplicated values')
print(df.loc[df.duplicated(keep = 'last')])

# Drop the rows with duplicated values
df3 = df2.drop_duplicates()

Sum of missing values: 0 

Duplicated values
Empty DataFrame
Columns: [web-scraper-order, web-scraper-start-url, rooms_area_price_raw, address_raw, price_raw, description_raw, text_raw, rooms, area, price, description_raw_len, text_raw_len, luxurious, area_cat, price_per_m2, datetime]
Index: []


### Save data to file

In [16]:
df3.to_csv('apartments_data_prepared.csv', 
          sep=",", 
          encoding='utf-8',
          index=False)

### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [17]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.5.0-1025-azure
Datetime: 2024-10-10 17:21:16
Python Version: 3.11.10
-----------------------------------
